In [14]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)

import shared_utils

In [2]:
from IPython.core.magic import register_cell_magic

In [3]:
## parameters cell
itp_id = 300

In [4]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [5]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [6]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
53,City of Santa Monica,300,recJHFDLpGRMIFgnL,07 - Los Angeles,True,2024-04-17,efbbd5293be71f7a5de0cf82b59febe1,not_yet_run


In [7]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [8]:
# speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')` #  non-aggregated

In [10]:
speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_speedmap_segments_{analysis_date}.parquet') #  aggregated

In [11]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'base64_url',
       'organization_source_record_id', 'organization_name',
       'caltrans_district', 'geometry'],
      dtype='object')

In [12]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [18]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,base64_url,organization_source_record_id,organization_name,caltrans_district,geometry
472062,efbbd5293be71f7a5de0cf82b59febe1,7952ac84db347ba2c6e65e60e5988275,26757,1640__1709,FLORENCE EB & LA CIENEGA NS__MESMER SB & JEFFE...,1640-1709-1,offpeak,9.91,15,9.88,9.94,Big Blue Bus Schedule,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,recJHFDLpGRMIFgnL,City of Santa Monica,07 - Los Angeles,"LINESTRING (150525.427 -448969.227, 150596.921..."
472063,efbbd5293be71f7a5de0cf82b59febe1,7952ac84db347ba2c6e65e60e5988275,26757,1640__1709,FLORENCE EB & LA CIENEGA NS__MESMER SB & JEFFE...,1640-1709-1,peak,6.09,15,6.05,6.12,Big Blue Bus Schedule,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,recJHFDLpGRMIFgnL,City of Santa Monica,07 - Los Angeles,"LINESTRING (150525.427 -448969.227, 150596.921..."
472064,efbbd5293be71f7a5de0cf82b59febe1,7952ac84db347ba2c6e65e60e5988275,26757,1640__1709,FLORENCE EB & LA CIENEGA NS__MESMER SB & JEFFE...,1640-1709-1,all_day,6.15,30,6.08,6.22,Big Blue Bus Schedule,aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vY3VycmVudC...,recJHFDLpGRMIFgnL,City of Santa Monica,07 - Los Angeles,"LINESTRING (150525.427 -448969.227, 150596.921..."


In [19]:
speedmap_segs.time_period.unique()

array(['offpeak', 'peak', 'all_day'], dtype=object)

In [20]:
peak_test = speedmap_segs >> filter(_.time_period == 'peak')

In [25]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [30]:
total_peak = len([x[1] for x in time_periods if 'Peak' in x[1]])

In [31]:
total_peak

8

In [55]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / total_peak

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    cmap = shared_utils.rt_utils.ZERO_THIRTY_COLORSCALE
    url = shared_utils.rt_utils.SPEEDMAP_LEGEND_URL

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [66]:
speedmap_state = shared_utils.rt_utils.set_state_export(peak_test, filename='testnew4', map_type='speedmap',
                                      color_col='p20_mph', cmap=cmap, legend_url=url,
                                              cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/testnew4.geojson.gz


In [67]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-04-17',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/testnew4.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.02642231679744, -118.44452237468757),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMDQtMTciLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdG5ldzQuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzQuMDI2NDIyMzE2Nzk3NDQsIC0xMTguNDQ0NTIyMzc0Njg3NTddLCAiem9vbSI6IDEzLCAibGVnZW5kX3V

In [56]:
standard_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_stop_segments_{analysis_date}.parquet') #  aggregated
standard_segs = standard_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])
standard_segs = standard_segs >> filter(_.time_period == 'peak')

In [57]:
standard_segs = prep_gdf(standard_segs)

In [64]:
standard_state = shared_utils.rt_utils.set_state_export(standard_segs, filename='testnew4standard', map_type='speedmap',
                                      color_col='p20_mph', cmap=cmap, legend_url=url,
                                              cache_seconds=0, map_title=f'Stop Segs {analysis_date}')

writing to calitp-map-tiles/testing/testnew4standard.geojson.gz


In [65]:
standard_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Stop Segs 2024-04-17',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/testnew4standard.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.02702541231935, -118.44722696700545),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTdG9wIFNlZ3MgMjAyNC0wNC0xNyIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy90ZXN0bmV3NHN0YW5kYXJkLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNSwgInRvb2x0aXBfc3BlZWRfa2V5IjogInAyMF9tcGgifSwgInR5cGUiOiAic3BlZWRtYXAifV0sICJsYXRfbG9uIjogWzM0LjAyNzAyNTQxMjMxOTM1LCAtMTE4LjQ0NzIyNjk2NzAwNTQ1XSwgInpvb20iOiAxMywgImx

## TODO replace status with a scan of speedmap segs column?